# 🌟 PROJETO ETL – Análise Automatizada de Feedbacks Internos (Versão Oficial - Karin)

### 📘 1. Introdução

Bem-vindo ao meu projeto ETL – Extração, Transformação e Carregamento de Feedbacks Internos, uma iniciativa criada para demonstrar como dados textuais não estruturados podem ser convertidos em informações úteis para decisões organizacionais.

Neste projeto:

- Eu simulo uma base de dados interna contendo feedbacks em texto.

- Construo um pipeline ETL completo sem depender de APIs.

- Transformo os textos brutos em métricas estruturadas:

  - limpeza linguística

  - remoção de stopwords

  - análise de sentimento simples

  - contagem de palavras

- Gero como saída CSV e JSON, prontos para BI, dashboards ou análises.

Este projeto é uma releitura criativa do processo ETL que aprendi anteriormente, adaptado para um novo domínio e com implementação 100% própria.

### 🧱 2. Arquitetura Geral do Projeto (ETL)

A solução é dividida em três módulos principais:

**E – Extração**

Carrego vários arquivos .txt contendo textos brutos armazenados em uma pasta.

**T – Transformação**

Aplico diversas etapas de pré-processamento:

- limpeza de caracteres

- padronização

- remoção de stopwords

- criação de variáveis (features)

- sentiment analysis usando regras simples

**L – Carregamento**

Salvo o dataset final em:

- feedbacks_processados.csv

- feedbacks_processados.json

## 🛠️ 3. Preparação do Ambiente

In [1]:
import os
import pandas as pd
import re
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

print("Ambiente inicializado!")

Ambiente inicializado!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 🧪 4. Criando Dados Fictícios

Para simular uma base interna de feedbacks, gero automaticamente três arquivos .txt em uma pasta de trabalho.

In [2]:
# Criando diretório de dados fictícios
os.makedirs("/content/dados_feedbacks", exist_ok=True)

# Conjunto fictício de feedbacks
feedbacks_ficticios = {
    "feedback1.txt": "O atendimento foi excelente, muito rápido e eficiente.",
    "feedback2.txt": "Achei o serviço ruim e demorado. Não gostei da experiência.",
    "feedback3.txt": "O produto é bom, mas poderia ser melhor. No geral, foi ok.",
}

# Gravando os arquivos
for nome, conteudo in feedbacks_ficticios.items():
    with open(f"/content/dados_feedbacks/{nome}", "w", encoding="utf-8") as arq:
        arq.write(conteudo)

print("Arquivos fictícios criados com sucesso!")

Arquivos fictícios criados com sucesso!


## 📥 5. ETAPA E – Extração

Carrego automaticamente todos os .txt da pasta.

In [3]:
def extrair_textos(pasta):
    linhas = []
    for arquivo in os.listdir(pasta):
        if arquivo.endswith(".txt"):
            with open(os.path.join(pasta, arquivo), "r", encoding="utf-8") as f:
                linhas.append({
                    "arquivo": arquivo,
                    "texto_original": f.read()
                })
    return pd.DataFrame(linhas)

df_raw = extrair_textos("/content/dados_feedbacks")
df_raw

,arquivo,texto_original
0,feedback2.txt,Achei o serviço ruim e demorado. Não gostei da...
1,feedback3.txt,"O produto é bom, mas poderia ser melhor. No ge..."
2,feedback1.txt,"O atendimento foi excelente, muito rápido e ef..."


## 🔧 6. ETAPA T – Transformação

### 6.1. Limpeza de texto

In [4]:
def limpar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"[^a-zA-Z0-9áéíóúãõç ]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

### 6.2. Remoção de Stopwords

In [5]:
stop_pt = set(stopwords.words("portuguese"))

def remover_stopwords(texto):
    palavras = texto.split()
    filtradas = [p for p in palavras if p not in stop_pt]
    return " ".join(filtradas)


### 6.3. Análise simples de sentimento

In [6]:
positivas = {"bom", "ótimo", "excelente", "maravilhoso", "gostei", "eficiente", "rápido"}
negativas = {"ruim", "péssimo", "horrível", "demorado", "não", "gostei"}

def classificar_sentimento(texto):
    tokens = set(texto.split())
    pos = len(tokens & positivas)
    neg = len(tokens & negativas)

    if pos > neg:
        return "positivo"
    elif neg > pos:
        return "negativo"
    else:
        return "neutro"

### 6.4. Aplicando todas as transformações

In [7]:
df = df_raw.copy()

df["texto_limpo"] = df["texto_original"].apply(limpar_texto)
df["sem_stopwords"] = df["texto_limpo"].apply(remover_stopwords)
df["sentimento"] = df["sem_stopwords"].apply(classificar_sentimento)
df["contagem_palavras"] = df["sem_stopwords"].str.split().str.len()

df

,arquivo,texto_original,texto_limpo,sem_stopwords,sentimento,contagem_palavras
0,feedback2.txt,Achei o serviço ruim e demorado. Não gostei da...,achei o serviço ruim e demorado não gostei da ...,achei serviço ruim demorado gostei experincia,negativo,6
1,feedback3.txt,"O produto é bom, mas poderia ser melhor. No ge...",o produto é bom mas poderia ser melhor no gera...,produto bom poderia melhor geral ok,positivo,6
2,feedback1.txt,"O atendimento foi excelente, muito rápido e ef...",o atendimento foi excelente muito rápido e efi...,atendimento excelente rápido eficiente,positivo,4


## 📤 7. ETAPA L – Carregamento

### 7.1. Exportando em CSV

In [8]:
df.to_csv("/content/feedbacks_processados.csv", index=False, encoding="utf-8")
print("Arquivo CSV gerado!")

Arquivo CSV gerado!


### 7.2. Exportando em JSON

In [9]:
df.to_json("/content/feedbacks_processados.json", orient="records", force_ascii=False)
print("Arquivo JSON gerado!")

Arquivo JSON gerado!
